In [849]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt
import sklearn.linear_model as linear_model

from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder

from scipy import stats
from scipy.stats import skew,norm  # for some statistics
from scipy.special import boxcox1p

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import  GradientBoostingClassifier,RandomForestClassifier

import lightgbm as lgbm
import xgboost as xgb
from catboost import CatBoostClassifier

from sklearn.linear_model import  LinearRegression, Ridge
import os, gc, warnings
warnings.filterwarnings('ignore')

In [850]:
ttrain = pd.read_csv('Train.csv')
ttest = pd.read_csv('Test.csv')
submission = pd.read_csv("SampleSubmission.csv")

In [851]:
fin_data = pd.concat([ttrain,ttest]).reset_index(drop=True)

In [852]:
fin_data.drop(['ID','LGA_Name'], axis =1, inplace = True)

In [853]:
list = []
for i in fin_data.columns:
    kd = fin_data[i].isna().sum()
    ml = len(fin_data[i])
    jd = kd/ml * 100
    
    
    if jd > 70:
        fin_data.drop(i, axis = 1, inplace = True)
        list.append(i)
        
        
        """ Dropping fetaures with missing values greater than 70% """

In [854]:
mapper = {'Joint Gender':'Other', 'NOT STATED':'Other', 'NO GENDER': 'Other', 'SEX':"Other"}
fin_data.Gender = fin_data.Gender.replace(mapper)

# Confirm mappings
fin_data.Gender.value_counts()

Male      8356
Female    3679
Other      546
Entity     300
Name: Gender, dtype: int64

In [855]:
## Mapping  by geopolitical zones

mapper = {'Ibeju-Lekki':'Geo1','Oshodi-Isolo':'Geo1','ABULE-EGBA':'Geo1','AJAO-ESTATE':'Geo1','Orile-Iganmu':'Geo1',
          'Ajeromi-Ifelodun':'Geo1','Ifako-Ijaye':'Geo1','Amuwo-Odofin':'Geo1','Eti-Osa':'Geo1','Ajegunle-State':'Geo1',
          'Ibadan-East':'Geo1','Ibadan-West':'Geo1', 'Ibadan-North':'Geo1','Oyo-East':'Geo1','Oyo':'Geo1',
          'Ogbmosho-South':'Geo1','Oyo-West':'Geo1', 'Lagelu-North':'Geo1','Ibarapa-Central':'Geo1','Lagos':'Geo1',
          'Ife-Central':'Geo1', 'Ife-North':'Geo1','Ilesha-West':'Geo1','Ilesha-East':'Geo1','Osun':'Geo1',
          'Ile-Oluji':'Geo1','Ondo_West':'Geo1','Ondo':'Geo1',
          'Obafemi-Owode':'Geo1','Ogun-Waterside':'Geo1','Ogun-Waterside':'Geo1','Ijebu-North':'Geo1','Ijebu-Ode':'Geo1',
          'Ijebu-East':'Geo1', 'Ado-Ota':'Geo1', 'Ogun':'Geo1',
          'Ado-Ekiti':'Geo1', 'Ekiti-East':'Geo1','Akoko-West':'Geo1','Ondo-West':'Geo1','Ekiti-West':'Geo1',
          
          
          'Onitsha-North':'Geo2', 'Onitsha-South':'Geo2','Anambra-East':'Geo2','Awka-North':'Geo2',
          'Awka-South':'Geo2','Idemili-south':'Geo2','Nnewi-South':'Geo2','Idemili-North':'Geo2','Anambra':'Geo2',
          'Ngor-Okpala':'Geo2','Owerri-North':'Geo2','Owerri-West':'Geo2','Owerri-Municipal':'Geo2',
          'Owerri-West':'Geo2','Imo':'Geo2','Aboh-Mbaise':'Geo2',
          'Umuahia-South':'Geo2', 'Aba-North':'Geo2','Aba-South':'Geo2','Abia':'Geo2','Udi-Agwu':'Geo2',
          'Enugu-North':'Geo2','ENUGU-EAST':'Geo2','ENUGU-SOUTH':'Geo2','Nnewi-North':'Geo2','Ebonyi':'Geo2',
          
          
          'Essien-Udim':'Geo3','Nsit-Ubium':'Geo3','Akwa-Ibom':'Geo3',
          'Obia-Akpor':'Geo3','Port-Harcourt':'Geo3','Asari-Toru':'Geo3','Calabar-Municipality':'Geo3',
          'Rivers':'Geo3','Cross-River':'Geo3', 'Ovia-SouthWest':'Geo3','Ogba-Ndoni':'Geo3',
          'Warri':'Geo3','Warri-Central':'Geo3','Warri-South':'Geo3','Oshimili-North':'Geo3',
          'Ethiope-East':'Geo3','Ughelli-North':'Geo3', 'Ndokwa-East':'Geo3','Isoko-south':'Geo3',
          'Isoko-North':'Geo3','Warri-North':'Geo3','Delta':'Geo3','Esan-Central':'Geo3','Aniocha-South':'Geo3',
          'Bayelsa':'Geo3','Esan-West':'Geo3','Edo':'Geo3','Etsako-West':'Geo3',
          
          
          'Abuja-Municipal':'Geo4','Central-Abuja':'Geo4','Abuja':'Geo4','QuaAn-Pan':'Geo4',
          'Ilorin-West':'Geo4','Ilorin-East':'Geo4','Kwara':'Geo4','Benue':'Geo4','Nasarawa':'Geo4','Kogi':'Geo4',
          'Niger':'Geo4','Jos-North':'Geo4','Jos-South':'Geo4','Niger-State':'Geo4',
          
          
          'Kaduna-South':'Geo5','Kaduna':'Geo5','Kano':'Geo5','Kaduna-North':'Geo5','Kano-Municipal':'Geo5','Kebbi':'Geo5',
          
          
          'Bauchi':'Geo6','Gombe':'Geo6',
          
          
          'N-A':'None'}

fin_data.State = fin_data.State.replace(mapper)

# Confirm mappings
fin_data.State.value_counts()

Geo1    4377
Geo4    1083
Geo3     451
Geo2     164
Geo5      57
Geo6       4
None       1
Name: State, dtype: int64

In [856]:
def fill(col):
    for i in col:
        #fin_data[i].fillna(fin_data[i].mode()[0], inplace = True)
        #fin_data[i].astype(str)
        fin_data[i].fillna('None', inplace = True)
        
        """ Filling up the remaining features with a very large number """
        
        #fin_data[i].mode()[0]

In [857]:
fill(fin_data.drop(["target","State",'Car_Category'],axis=1))

In [858]:
fin_data["State"].fillna('None', inplace = True)
fin_data["Car_Category"].fillna('None', inplace = True)

In [859]:
dcol = [col for col in fin_data.columns if 'Date' in col]

In [860]:
for col in dcol:
    fin_data[col] = pd.to_datetime(fin_data[col])

In [861]:
for col in dcol:
  for date_feature in ['year', 'month', 'day']:
    fin_data[col+date_feature] = getattr(fin_data[col].dt, date_feature)

fin_data.head()

,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,State,...,target,Policy Start Dateyear,Policy Start Datemonth,Policy Start Dateday,Policy End Dateyear,Policy End Datemonth,Policy End Dateday,First Transaction Dateyear,First Transaction Datemonth,First Transaction Dateday
0,2010-05-14,2011-05-13,Male,30,2010-05-14,1,Saloon,Black,TOYOTA,None,...,0.0,2010,5,14,2011,5,13,2010,5,14
1,2010-11-29,2011-11-28,Female,79,2010-11-29,1,JEEP,Grey,TOYOTA,None,...,1.0,2010,11,29,2011,11,28,2010,11,29
2,2010-03-21,2011-03-20,Male,43,2010-03-21,1,Saloon,Red,TOYOTA,None,...,0.0,2010,3,21,2011,3,20,2010,3,21
3,2010-08-21,2011-08-20,Male,2,2010-08-21,1,None,None,None,None,...,0.0,2010,8,21,2011,8,20,2010,8,21
4,2010-08-29,2010-12-31,Entity,20,2010-08-29,3,None,None,None,Geo1,...,1.0,2010,8,29,2010,12,31,2010,8,29


In [862]:
fin_data["Policy_Month_Diff"] = abs(fin_data['Policy End Datemonth'] - fin_data['Policy Start Datemonth'])
fin_data['Policy_Day_Diff'] = abs(fin_data['Policy End Date'] - fin_data['Policy Start Date']).dt.days
fin_data['Policy_Week_Diff'] = abs(round(fin_data['Policy_Day_Diff']/7, 0))

In [863]:
fin_data.drop(['Policy Start Date','Policy End Date','First Transaction Date'], axis =1, inplace = True)
fin_data.drop(['Policy Start Dateyear','Policy End Dateyear','First Transaction Dateyear'], axis =1, inplace = True)
fin_data.drop(['First Transaction Datemonth'], axis =1, inplace = True)
fin_data.drop(['First Transaction Dateday'], axis =1, inplace = True)

In [864]:
fin_data.drop(['Policy_Day_Diff'], axis =1, inplace = True)

In [865]:
## drop policy end date day and policy end datemonth since we have taken the difference already
fin_data.drop(['Policy End Dateday','Policy End Datemonth'], axis =1, inplace = True)

In [866]:
listt = []
for i in range(len(fin_data['Age'])):
    if fin_data['Age'][i] < 0:
        fin_data['Age'][i] = abs(fin_data['Age'][i])

In [867]:
fin_data['Age'].unique()

array([  30,   79,   43,    2,   20,   37,   40,   71,   58,   45,   51,
        120,   38,   81,   69,   31,   70,   34,    1,   41,   47,   36,
         46,   48,   39,   63,   42,   27,   64,   50,   53,   33,   23,
         32,   59,   60,   52,   54,   44,   11,   78,   28,   29,   57,
         12,   65,   35,   73,   56,   55,   26,   49,   18,   14,   62,
         25,   61,   72,   82,   67,    7,   10,   68,   21,   76,   24,
         66,    3,   93,   22,   77,   19,   80,    9,    6,   89,   74,
         75,   84,   16,   90,  140,    8,    5,   13,   86,   15,  320,
         83,   85, 6099, 5939,    4,  144,  112,   17,    0,   88,   87,
        133,  102,  128,  100], dtype=int64)

In [868]:
fin_data['Age'][fin_data['Age']==6099] = fin_data['Age'].mode()[0]
fin_data['Age'][fin_data['Age']==5939] = fin_data['Age'].mode()[0]

In [869]:
fin_data['Age'][fin_data['Age']==6099]

Series([], Name: Age, dtype: int64)

In [870]:
numerical = fin_data.drop(['target'], axis =1).dtypes[fin_data.drop(['target'], axis =1).dtypes != 'object'].index

skewness = fin_data.drop(['target'], axis =1)[numerical].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)

skewness_df = pd.DataFrame({'Skewness': skewness})
skewness_df.head(10)

,Skewness
Policy_Week_Diff,44.518024
No_Pol,3.772737
Age,3.278167
Policy_Month_Diff,3.210912
Policy Start Datemonth,0.030754
Policy Start Dateday,-0.047862


In [871]:
from scipy.special import boxcox1p

boxcox_features = skewness_df[np.abs(skewness_df['Skewness'])>0.75].index
lam = 0.25

for col in boxcox_features:
    fin_data.loc[:, col] = boxcox1p(fin_data[col], lam)

In [872]:
numerical = fin_data.drop(['target'], axis =1).dtypes[fin_data.drop(['target'], axis =1).dtypes != 'object'].index

skewness = fin_data.drop(['target'], axis =1)[numerical].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)

skewness_df = pd.DataFrame({'Skewness': skewness})
skewness_df.head(10)

,Skewness
Policy_Month_Diff,2.623812
No_Pol,2.484713
Policy_Week_Diff,0.550963
Policy Start Datemonth,0.030754
Policy Start Dateday,-0.047862
Age,-1.356334


In [873]:
fin_data.drop(['Subject_Car_Colour'], axis =1, inplace = True)

In [874]:
fin_data.drop(['Subject_Car_Make'], axis=1, inplace = True)

In [875]:
"""Regrouping the Car_Category feature"""

mapper = {'Mini Van':'Other','Pick Up':'Other','Sedan':'Other','Mini Van':'Other',
          'Tipper Truck':'Other',"Van":'Other',"Bus":'Other',"Truck":"Other",
         "Mini Bus":'Other', "Wagon": 'Other',"Shape Of Vehicle Chasis":'Other', 
         "Station 4 Wheel": 'Other', "Pick Up > 3 Tons": 'Other', "CAMRY CAR HIRE": 'Other',"Motorcycle": 'Other'}
fin_data.Car_Category = fin_data.Car_Category.replace(mapper)

# Confirm mappings
fin_data.Car_Category.value_counts()

Saloon    6633
None      4110
JEEP      2223
Other      315
Name: Car_Category, dtype: int64

In [876]:
## intution: join the "other" and "none" category together

mapper = {"None": 'Other'}
fin_data.Car_Category = fin_data.Car_Category.replace(mapper)

# Confirm mappings
fin_data.Car_Category.value_counts()

Saloon    6633
Other     4425
JEEP      2223
Name: Car_Category, dtype: int64

In [877]:
mapper = {"Car Vintage": 'Car Classic',"Motor Cycle": "Customized Motor"}
fin_data.ProductName = fin_data.ProductName.replace(mapper)

# Confirm mappings
fin_data.ProductName.value_counts()

Car Classic         7147
CarSafe             4154
Customized Motor     654
Car Plus             523
CVTP                 509
CarFlex              194
Muuve                100
Name: ProductName, dtype: int64

In [878]:
mapper = {"CVTP": 'Other',"CarFlex": "Other","Muuve": "Other","Car Plus": "Other","Customized Motor":"Other"}
fin_data.ProductName = fin_data.ProductName.replace(mapper)

# Confirm mappings
fin_data.ProductName.value_counts()

Car Classic    7147
CarSafe        4154
Other          1980
Name: ProductName, dtype: int64

In [879]:
# Label encoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
import pandas as pd
# Create new features
#fin_data['ProductName'] = le.fit_transform(fin_data['ProductName'])
#fin_data['Car_Category'] = le.fit_transform(fin_data['Car_Category'])
#fin_data['State'] = le.fit_transform(fin_data['State'])
#fin_data['Gender'] = le.fit_transform(fin_data['State'])

fin_data = pd.get_dummies(fin_data, columns=['Gender'], drop_first=True)


In [880]:
fin_data = pd.get_dummies(fin_data, columns=['ProductName'], drop_first=True)

In [881]:
fin_data

,Age,No_Pol,Car_Category,State,target,Policy Start Datemonth,Policy Start Dateday,Policy_Month_Diff,Policy_Week_Diff,Gender_Female,Gender_Male,Gender_None,Gender_Other,ProductName_CarSafe,ProductName_Other
0,5.438444,0.756828,Saloon,None,0.0,5,14,0.000000,6.792672,0,1,0,0,0,0
1,7.962790,0.756828,JEEP,None,1.0,11,29,0.000000,6.792672,1,0,0,0,0,0
2,6.302038,0.756828,Saloon,None,0.0,3,21,0.000000,6.792672,0,1,0,0,0,0
3,1.264296,0.756828,Other,None,0.0,8,21,0.000000,6.792672,0,1,0,0,1,0
4,4.562781,1.656854,Other,Geo1,1.0,8,29,1.981395,4.351191,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13276,7.486487,1.264296,JEEP,Geo1,NaN,12,5,0.000000,6.792672,0,1,0,0,0,0
13277,6.302038,0.756828,Saloon,Geo1,NaN,1,14,0.000000,6.792672,0,1,0,0,0,0
13278,5.438444,0.756828,Other,Geo1,NaN,7,26,0.000000,6.792672,0,1,0,0,0,1
13279,6.360080,1.264296,Saloon,Geo2,NaN,2,16,0.000000,6.792672,0,1,0,0,0,0


In [882]:
#fin_data.drop(['Policy_Month_Diff'], axis =1, inplace = True)

In [883]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits = 8, shuffle = True, random_state=99)

In [884]:
split=ttrain.shape[0]
train_df=fin_data[:split]
test_df=fin_data[split:]

In [885]:
def test_mean_target_encoding(train, test, target, categorical, alpha=5):
    # Calculate global mean on the train data
    global_mean = train[target].mean()
    
    # Group by the categorical feature and calculate its properties
    train_groups = train.groupby(categorical)
    category_sum = train_groups[target].sum()
    category_size = train_groups.size()
    
    # Calculate smoothed mean target statistics
    train_statistics = (category_sum + global_mean * alpha) / (category_size + alpha)
    
    # Apply statistics to the test data and fill new categories
    test_feature = test[categorical].map(train_statistics).fillna(global_mean)
    return test_feature.values

In [886]:
def train_mean_target_encoding(train, target, categorical, alpha=5):
    # Create 5-fold cross-validation
    kf = KFold(n_splits = 5, shuffle = True, random_state=99)
    train_feature = pd.Series(index=train.index)
    
    # For each folds split
    for train_index, test_index in kf.split(train):
        cv_train, cv_test = train.iloc[train_index], train.iloc[test_index]
      
        # Calculate out-of-fold statistics and apply to cv_test
        cv_test_feature = test_mean_target_encoding(cv_train, cv_test, target, categorical, alpha)
        
        # Save new feature for this particular fold
        train_feature.iloc[test_index] = cv_test_feature       
    return train_feature.values

In [887]:
def mean_target_encoding(train, test, target, categorical, alpha=5):
  
    # Get the train feature
    train_feature = train_mean_target_encoding(train, target, categorical, alpha)
  
    # Get the test feature
    test_feature = test_mean_target_encoding(train, test, target, categorical, alpha)
    
    # Return new features to add to the model
    return train_feature, test_feature

In [888]:
train_feat, test_feat = mean_target_encoding(train_df, test_df, 'target', 'Car_Category', alpha=5)

In [889]:
train_df['Car_Category'] = train_feat
test_df['Car_Category'] = test_feat

In [890]:
train_feat2, test_feat2 = mean_target_encoding(train_df, test_df, 'target', 'State', alpha=5)

In [891]:
train_df['State'] = train_feat2
test_df['State'] = test_feat2

In [892]:
#train_feat3, test_feat3 = mean_target_encoding(train_df, test_df, 'target', 'ProductName', alpha=5)

In [893]:
#train_df['ProductName'] = train_feat3
#test_df['ProductName'] = test_feat3

In [894]:
#train_feat4, test_feat4 = mean_target_encoding(train_df, test_df, 'target', 'Gender', alpha=5)

In [895]:
#train_df['Gender'] = train_feat4
#test_df['Gender'] = test_feat4

In [896]:
X = train_df.drop(['target'], axis =1)
y = train_df['target']

In [897]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
#ros = RandomOverSampler()
#X, y = ros.fit_resample(X, y)

from imblearn.over_sampling import SMOTE
over = SMOTE(sampling_strategy = 'minority', random_state=7)
X, y = over.fit_resample(X, y)


from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import BorderlineSMOTE
#ros = RandomOverSampler()
#X, y = ros.fit_resample(X, y)


In [845]:
under = RandomUnderSampler(sampling_strategy=1)

# X, y = under.fit_resample(X, y)

In [898]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25,random_state=19)

In [899]:
from catboost import Pool
cat_model= CatBoostClassifier()
cat_model.fit(Pool(X_train,y_train))
cat = cat_model.predict(X_test)

Learning rate set to 0.033601
0:	learn: 0.6643894	total: 12.1ms	remaining: 12.1s
1:	learn: 0.6383095	total: 24ms	remaining: 12s
2:	learn: 0.6178210	total: 36.6ms	remaining: 12.2s
3:	learn: 0.5996998	total: 48.2ms	remaining: 12s
4:	learn: 0.5796064	total: 59.6ms	remaining: 11.9s
5:	learn: 0.5686583	total: 71.4ms	remaining: 11.8s
6:	learn: 0.5532169	total: 82.4ms	remaining: 11.7s
7:	learn: 0.5438273	total: 93.6ms	remaining: 11.6s
8:	learn: 0.5331225	total: 105ms	remaining: 11.5s
9:	learn: 0.5264323	total: 116ms	remaining: 11.5s
10:	learn: 0.5197622	total: 127ms	remaining: 11.4s
11:	learn: 0.5108591	total: 139ms	remaining: 11.4s
12:	learn: 0.5056409	total: 150ms	remaining: 11.4s
13:	learn: 0.4982682	total: 162ms	remaining: 11.4s
14:	learn: 0.4921231	total: 173ms	remaining: 11.4s
15:	learn: 0.4881340	total: 191ms	remaining: 11.8s
16:	learn: 0.4845754	total: 205ms	remaining: 11.9s
17:	learn: 0.4796624	total: 217ms	remaining: 11.9s
18:	learn: 0.4751485	total: 240ms	remaining: 12.4s
19:	learn

164:	learn: 0.2655242	total: 2.32s	remaining: 11.7s
165:	learn: 0.2653550	total: 2.33s	remaining: 11.7s
166:	learn: 0.2651216	total: 2.36s	remaining: 11.8s
167:	learn: 0.2645755	total: 2.37s	remaining: 11.7s
168:	learn: 0.2643694	total: 2.38s	remaining: 11.7s
169:	learn: 0.2635545	total: 2.4s	remaining: 11.7s
170:	learn: 0.2632990	total: 2.41s	remaining: 11.7s
171:	learn: 0.2629461	total: 2.43s	remaining: 11.7s
172:	learn: 0.2622751	total: 2.44s	remaining: 11.7s
173:	learn: 0.2615472	total: 2.45s	remaining: 11.7s
174:	learn: 0.2613872	total: 2.47s	remaining: 11.7s
175:	learn: 0.2602747	total: 2.49s	remaining: 11.7s
176:	learn: 0.2600655	total: 2.51s	remaining: 11.7s
177:	learn: 0.2595238	total: 2.52s	remaining: 11.6s
178:	learn: 0.2586678	total: 2.53s	remaining: 11.6s
179:	learn: 0.2578927	total: 2.54s	remaining: 11.6s
180:	learn: 0.2572591	total: 2.56s	remaining: 11.6s
181:	learn: 0.2567722	total: 2.57s	remaining: 11.5s
182:	learn: 0.2564301	total: 2.58s	remaining: 11.5s
183:	learn: 0

327:	learn: 0.2110394	total: 4.61s	remaining: 9.45s
328:	learn: 0.2104938	total: 4.63s	remaining: 9.43s
329:	learn: 0.2103895	total: 4.64s	remaining: 9.41s
330:	learn: 0.2100991	total: 4.66s	remaining: 9.41s
331:	learn: 0.2099026	total: 4.67s	remaining: 9.39s
332:	learn: 0.2094264	total: 4.68s	remaining: 9.38s
333:	learn: 0.2091175	total: 4.69s	remaining: 9.36s
334:	learn: 0.2088381	total: 4.7s	remaining: 9.34s
335:	learn: 0.2087231	total: 4.72s	remaining: 9.32s
336:	learn: 0.2086135	total: 4.73s	remaining: 9.3s
337:	learn: 0.2084740	total: 4.74s	remaining: 9.29s
338:	learn: 0.2081433	total: 4.75s	remaining: 9.27s
339:	learn: 0.2079279	total: 4.77s	remaining: 9.25s
340:	learn: 0.2077591	total: 4.78s	remaining: 9.23s
341:	learn: 0.2075810	total: 4.79s	remaining: 9.22s
342:	learn: 0.2074631	total: 4.81s	remaining: 9.22s
343:	learn: 0.2073151	total: 4.83s	remaining: 9.21s
344:	learn: 0.2071756	total: 4.84s	remaining: 9.2s
345:	learn: 0.2070848	total: 4.86s	remaining: 9.18s
346:	learn: 0.2

493:	learn: 0.1840906	total: 6.75s	remaining: 6.91s
494:	learn: 0.1840023	total: 6.77s	remaining: 6.91s
495:	learn: 0.1839303	total: 6.78s	remaining: 6.89s
496:	learn: 0.1837125	total: 6.79s	remaining: 6.88s
497:	learn: 0.1835934	total: 6.8s	remaining: 6.86s
498:	learn: 0.1834831	total: 6.82s	remaining: 6.84s
499:	learn: 0.1833524	total: 6.83s	remaining: 6.83s
500:	learn: 0.1832285	total: 6.84s	remaining: 6.81s
501:	learn: 0.1831176	total: 6.85s	remaining: 6.8s
502:	learn: 0.1829606	total: 6.86s	remaining: 6.78s
503:	learn: 0.1828677	total: 6.88s	remaining: 6.76s
504:	learn: 0.1827801	total: 6.89s	remaining: 6.75s
505:	learn: 0.1826495	total: 6.9s	remaining: 6.74s
506:	learn: 0.1825211	total: 6.91s	remaining: 6.72s
507:	learn: 0.1824431	total: 6.93s	remaining: 6.71s
508:	learn: 0.1823101	total: 6.94s	remaining: 6.7s
509:	learn: 0.1821581	total: 6.95s	remaining: 6.68s
510:	learn: 0.1820394	total: 6.97s	remaining: 6.67s
511:	learn: 0.1819339	total: 6.98s	remaining: 6.65s
512:	learn: 0.18

658:	learn: 0.1678524	total: 8.88s	remaining: 4.59s
659:	learn: 0.1677690	total: 8.89s	remaining: 4.58s
660:	learn: 0.1677228	total: 8.9s	remaining: 4.57s
661:	learn: 0.1676685	total: 8.91s	remaining: 4.55s
662:	learn: 0.1675904	total: 8.93s	remaining: 4.54s
663:	learn: 0.1675148	total: 8.95s	remaining: 4.53s
664:	learn: 0.1674615	total: 8.96s	remaining: 4.51s
665:	learn: 0.1673485	total: 8.98s	remaining: 4.5s
666:	learn: 0.1671791	total: 8.99s	remaining: 4.49s
667:	learn: 0.1671242	total: 9s	remaining: 4.47s
668:	learn: 0.1670725	total: 9.01s	remaining: 4.46s
669:	learn: 0.1669606	total: 9.03s	remaining: 4.45s
670:	learn: 0.1668853	total: 9.04s	remaining: 4.43s
671:	learn: 0.1668207	total: 9.06s	remaining: 4.42s
672:	learn: 0.1667610	total: 9.08s	remaining: 4.41s
673:	learn: 0.1667138	total: 9.09s	remaining: 4.4s
674:	learn: 0.1666659	total: 9.1s	remaining: 4.38s
675:	learn: 0.1665988	total: 9.11s	remaining: 4.37s
676:	learn: 0.1665210	total: 9.12s	remaining: 4.35s
677:	learn: 0.16643

819:	learn: 0.1559667	total: 11.2s	remaining: 2.46s
820:	learn: 0.1558632	total: 11.2s	remaining: 2.44s
821:	learn: 0.1558084	total: 11.2s	remaining: 2.43s
822:	learn: 0.1556762	total: 11.2s	remaining: 2.42s
823:	learn: 0.1556248	total: 11.3s	remaining: 2.4s
824:	learn: 0.1555769	total: 11.3s	remaining: 2.39s
825:	learn: 0.1555102	total: 11.3s	remaining: 2.38s
826:	learn: 0.1554052	total: 11.3s	remaining: 2.36s
827:	learn: 0.1553446	total: 11.3s	remaining: 2.35s
828:	learn: 0.1553073	total: 11.3s	remaining: 2.34s
829:	learn: 0.1552310	total: 11.3s	remaining: 2.32s
830:	learn: 0.1551511	total: 11.4s	remaining: 2.31s
831:	learn: 0.1550814	total: 11.4s	remaining: 2.3s
832:	learn: 0.1550346	total: 11.4s	remaining: 2.29s
833:	learn: 0.1549964	total: 11.4s	remaining: 2.27s
834:	learn: 0.1549586	total: 11.4s	remaining: 2.26s
835:	learn: 0.1548969	total: 11.5s	remaining: 2.25s
836:	learn: 0.1548603	total: 11.5s	remaining: 2.23s
837:	learn: 0.1547281	total: 11.5s	remaining: 2.22s
838:	learn: 0.

986:	learn: 0.1459549	total: 13.7s	remaining: 180ms
987:	learn: 0.1459002	total: 13.7s	remaining: 166ms
988:	learn: 0.1458276	total: 13.7s	remaining: 152ms
989:	learn: 0.1457735	total: 13.7s	remaining: 139ms
990:	learn: 0.1457222	total: 13.7s	remaining: 125ms
991:	learn: 0.1455915	total: 13.8s	remaining: 111ms
992:	learn: 0.1455523	total: 13.8s	remaining: 97ms
993:	learn: 0.1454740	total: 13.8s	remaining: 83.1ms
994:	learn: 0.1453690	total: 13.8s	remaining: 69.3ms
995:	learn: 0.1452960	total: 13.8s	remaining: 55.4ms
996:	learn: 0.1452684	total: 13.8s	remaining: 41.6ms
997:	learn: 0.1452484	total: 13.8s	remaining: 27.7ms
998:	learn: 0.1452207	total: 13.8s	remaining: 13.9ms
999:	learn: 0.1451853	total: 13.9s	remaining: 0us


In [900]:
from sklearn.metrics import f1_score
print(f1_score(y_test,cat))
#print(classification_report(y_test,cat))

0.9219716623428458


In [901]:
predictions = cat_model.predict(test_df.drop(['target'],axis =1))
sub= ttest[["ID"]]
sub['target'] = predictions
sub.to_csv('catuu.csv', index = False)

In [902]:
sub['target'].unique()

array([1., 0.])

In [903]:
def model(model,train,label,test,model_name):
    mean_train = []
    mean_test = []
    test_pred = np.zeros(test.shape[0])
    val_pred = np.zeros(train.shape[0])
    for count, (train_index,test_index) in enumerate(skf.split(train,label)):
        x_train,x_test = train.iloc[train_index],train.iloc[test_index]
        y_train,y_test = label.iloc[train_index],label.iloc[test_index]
        print(f'========================Fold{count +1}==========================')
        model.fit(x_train, y_train)
        train_predict = model.predict(x_train)
        test_predict = model.predict(x_test)
        val_pred[test_index] = test_predict
        test_pred+= model.predict(test)
        
        print('\nValidation scores', f1_score(y_test,test_predict))
        print('\nTraining scores', f1_score(y_train,train_predict))
        mean_train.append(f1_score(y_train, train_predict))
        mean_test.append(f1_score(y_test,test_predict))
    print('Average Testing ROC score for 10 folds split:',np.mean(mean_test))
    print('Average Training ROC score for 10 folds split:',np.mean(mean_train))
    print('standard Deviation for 10 folds split:',np.std(mean_test))
    return val_pred, test_pred, model_name
        
def Stacking(model,Train_stack,Test_stack,target,file_name):
    prediction = model.fit(Train_stack, target).predict(Test_stack)
    submission['target'] = prediction
    submission.to_csv(file_name,index=False)

In [904]:
def lgb(model,train,label,test,model_name):
    mean_train = []
    mean_test = []
    test_pred = np.zeros(test.shape[0])
    val_pred = np.zeros(train.shape[0])
    for count, (train_index,test_index) in enumerate(skf.split(train,label)):
        x_train,x_test = train.iloc[train_index],train.iloc[test_index]
        y_train,y_test = label.iloc[train_index],label.iloc[test_index]
        print(f'========================Fold{count +1}==========================')
        model.fit(x_train,y_train,eval_set=[(x_test,y_test)],early_stopping_rounds=500,
                               verbose=250)
        train_predict = model.predict(x_train, num_iteration = model.best_iteration_)
        test_predict = model.predict(x_test, num_iteration = model.best_iteration_)
        val_pred[test_index] = test_predict
        test_pred+= model.predict(test, num_iteration = model.best_iteration_)
        
        print('\nValidation scores', f1_score(y_test,test_predict))
        print('\nTraining scores', f1_score(y_train,train_predict))
        mean_train.append(f1_score(y_train, train_predict))
        mean_test.append(f1_score(y_test,test_predict))
    print('Average F1 score for 10 folds split:',np.mean(mean_test))
    print('Average Training ROC score for 10 folds split:',np.mean(mean_train))
    print('standard Deviation for 10 folds split:',np.std(mean_test))
    return val_pred, test_pred, model_name

In [905]:
lgb_model = lgbm.LGBMClassifier(random_state=34, n_estimators=8000, num_leaves=25,reg_lambda=6,reg_alpha=6 ,metric='AUC', learning_rate=0.5,
    max_depth=8, class_weight = 'balanced')
LGB1_train, LGB1_test, LGB1_name =lgb(lgb_model,X, y, test_df.drop(['target'], axis =1),'lightgbm(1)')

========================Fold1==========================
Training until validation scores don't improve for 500 rounds
[250]	valid_0's auc: 0.970902
[500]	valid_0's auc: 0.970902
Early stopping, best iteration is:
[86]	valid_0's auc: 0.970964

Validation scores 0.9172576832151301

Training scores 0.9420005603810591
========================Fold2==========================
Training until validation scores don't improve for 500 rounds
[250]	valid_0's auc: 0.970638
[500]	valid_0's auc: 0.970638
Early stopping, best iteration is:
[91]	valid_0's auc: 0.970791

Validation scores 0.9207221350078493

Training scores 0.9417752670731027
========================Fold3==========================
Training until validation scores don't improve for 500 rounds
[250]	valid_0's auc: 0.973628
[500]	valid_0's auc: 0.973628
Early stopping, best iteration is:
[87]	valid_0's auc: 0.973888

Validation scores 0.926771653543307

Training scores 0.9411698789780368
========================Fold4========================

In [906]:
LGB1_test

array([8., 8., 8., ..., 8., 8., 8.])

In [907]:
def cat(model,train,label,test,model_name):
    mean_train = []
    mean_test = []
    test_pred = np.zeros(test.shape[0])
    val_pred = np.zeros(train.shape[0])
    for count, (train_index,test_index) in enumerate(skf.split(train,label)):
        x_train,x_test = train.iloc[train_index],train.iloc[test_index]
        y_train,y_test = label.iloc[train_index],label.iloc[test_index]
        print(f'========================Fold{count +1}==========================')
        model.fit(x_train,y_train,eval_set=[(x_test,y_test)],early_stopping_rounds=500,
                           verbose=250,use_best_model=True)
        train_predict = model.predict(x_train)
        test_predict = model.predict(x_test)
        val_pred[test_index] = test_predict
        test_pred+= model.predict(test)
        print('\nTesting scores', f1_score(y_test,test_predict))
        print('\nTraining scores', f1_score(y_train,train_predict))
        mean_train.append(f1_score(y_train, train_predict))
        mean_test.append(f1_score(y_test,test_predict))
    print('Average Testing ROC score for 8 folds split:',np.mean(mean_test))
    print('Average Training ROC score for 8 folds split:',np.mean(mean_train))
    print('standard Deviation for 8 folds split:',np.std(mean_test))
    return val_pred, test_pred, model_name

In [908]:
catboost2 =  CatBoostClassifier(random_seed=34,bootstrap_type='Bayesian',max_depth=6,learning_rate=0.007,
                          iterations=12000,silent=True,eval_metric='AUC')


cat2_train, cat2_test, cat2_name = cat(catboost2,X,y,test_df.drop(['target'], axis =1),'catboost(2)')

========================Fold1==========================
0:	test: 0.8121674	best: 0.8121674 (0)	total: 15.5ms	remaining: 3m 6s
250:	test: 0.9497211	best: 0.9497211 (250)	total: 3.76s	remaining: 2m 55s
500:	test: 0.9592273	best: 0.9592273 (500)	total: 7.26s	remaining: 2m 46s
750:	test: 0.9624563	best: 0.9624563 (750)	total: 10.9s	remaining: 2m 42s
1000:	test: 0.9646150	best: 0.9646150 (1000)	total: 14.5s	remaining: 2m 39s
1250:	test: 0.9664646	best: 0.9664646 (1250)	total: 17.9s	remaining: 2m 33s
1500:	test: 0.9678266	best: 0.9678266 (1500)	total: 21.3s	remaining: 2m 29s
1750:	test: 0.9690281	best: 0.9690281 (1750)	total: 24.8s	remaining: 2m 25s
2000:	test: 0.9698622	best: 0.9698996 (1982)	total: 28.3s	remaining: 2m 21s
2250:	test: 0.9705614	best: 0.9705653 (2248)	total: 32.3s	remaining: 2m 19s
2500:	test: 0.9709900	best: 0.9709900 (2500)	total: 35.5s	remaining: 2m 14s
2750:	test: 0.9714017	best: 0.9714091 (2742)	total: 38.8s	remaining: 2m 10s
3000:	test: 0.9717595	best: 0.9717606 (2996)

750:	test: 0.9597343	best: 0.9597377 (749)	total: 10.1s	remaining: 2m 31s
1000:	test: 0.9628688	best: 0.9628688 (1000)	total: 13.5s	remaining: 2m 28s
1250:	test: 0.9654204	best: 0.9654204 (1250)	total: 16.9s	remaining: 2m 24s
1500:	test: 0.9675133	best: 0.9675133 (1500)	total: 20.3s	remaining: 2m 22s
1750:	test: 0.9690250	best: 0.9690250 (1750)	total: 23.7s	remaining: 2m 18s
2000:	test: 0.9701795	best: 0.9701795 (2000)	total: 27s	remaining: 2m 14s
2250:	test: 0.9709121	best: 0.9709223 (2248)	total: 30.4s	remaining: 2m 11s
2500:	test: 0.9716265	best: 0.9716328 (2498)	total: 33.8s	remaining: 2m 8s
2750:	test: 0.9720161	best: 0.9720161 (2750)	total: 37.2s	remaining: 2m 4s
3000:	test: 0.9723863	best: 0.9724107 (2948)	total: 40.6s	remaining: 2m 1s
3250:	test: 0.9725842	best: 0.9725899 (3243)	total: 44.4s	remaining: 1m 59s
3500:	test: 0.9728513	best: 0.9728558 (3493)	total: 48.3s	remaining: 1m 57s
3750:	test: 0.9730061	best: 0.9730373 (3707)	total: 51.6s	remaining: 1m 53s
4000:	test: 0.97306

3000:	test: 0.9747905	best: 0.9748053 (2998)	total: 40.9s	remaining: 2m 2s
3250:	test: 0.9750191	best: 0.9750298 (3246)	total: 44.3s	remaining: 1m 59s
3500:	test: 0.9752878	best: 0.9752941 (3481)	total: 47.6s	remaining: 1m 55s
3750:	test: 0.9754284	best: 0.9754284 (3750)	total: 51s	remaining: 1m 52s
4000:	test: 0.9755538	best: 0.9755543 (3996)	total: 54.4s	remaining: 1m 48s
4250:	test: 0.9755764	best: 0.9755764 (4250)	total: 57.7s	remaining: 1m 45s
4500:	test: 0.9756354	best: 0.9756570 (4483)	total: 1m 1s	remaining: 1m 42s
4750:	test: 0.9756819	best: 0.9756989 (4722)	total: 1m 4s	remaining: 1m 39s
5000:	test: 0.9757052	best: 0.9757403 (4881)	total: 1m 8s	remaining: 1m 35s
5250:	test: 0.9757789	best: 0.9757998 (5206)	total: 1m 11s	remaining: 1m 32s
5500:	test: 0.9758169	best: 0.9758424 (5347)	total: 1m 15s	remaining: 1m 28s
5750:	test: 0.9758106	best: 0.9758549 (5544)	total: 1m 18s	remaining: 1m 25s
6000:	test: 0.9757636	best: 0.9758549 (5544)	total: 1m 21s	remaining: 1m 21s
Stopped by 

In [909]:
catboost =  CatBoostClassifier(random_seed=34,use_best_model=True,
                          n_estimators=12000,silent=True,eval_metric='AUC', loss_function = 'CrossEntropy')

cat1_train, cat1_test, cat1_name = cat(catboost,X,y,test_df,'catboost(1)')

========================Fold1==========================
0:	test: 0.8097014	best: 0.8097014 (0)	total: 27.2ms	remaining: 5m 26s
250:	test: 0.9659103	best: 0.9659103 (250)	total: 3.54s	remaining: 2m 45s
500:	test: 0.9704270	best: 0.9704270 (500)	total: 6.91s	remaining: 2m 38s
750:	test: 0.9720821	best: 0.9720838 (748)	total: 10.3s	remaining: 2m 33s
1000:	test: 0.9730795	best: 0.9731062 (985)	total: 13.6s	remaining: 2m 29s
1250:	test: 0.9736534	best: 0.9736551 (1249)	total: 17.1s	remaining: 2m 26s
1500:	test: 0.9741087	best: 0.9741087 (1499)	total: 20.4s	remaining: 2m 22s
1750:	test: 0.9742556	best: 0.9742935 (1721)	total: 23.7s	remaining: 2m 18s
2000:	test: 0.9746746	best: 0.9746774 (1998)	total: 27.1s	remaining: 2m 15s
2250:	test: 0.9747319	best: 0.9748487 (2144)	total: 30.5s	remaining: 2m 11s
2500:	test: 0.9747449	best: 0.9748487 (2144)	total: 34.1s	remaining: 2m 9s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.9748486605
bestIteration = 2144

Shrink model to fir

In [910]:
gbm_model = GradientBoostingClassifier(max_depth=5,min_samples_leaf=9,n_estimators=3000,learning_rate=0.6,min_samples_split=10,random_state=10)
gbm_train, gbm_test, gbm_name= model(gbm_model,X,y,test_df.drop(['target'], axis =1), 'Gbm')

========================Fold1==========================

Validation scores 0.9102072141212585

Training scores 0.9927933742067334
========================Fold2==========================

Validation scores 0.9105382865807431

Training scores 0.9926363880677237
========================Fold3==========================

Validation scores 0.9115749525616698

Training scores 0.9926858126277295
========================Fold4==========================

Validation scores 0.9133918351774132

Training scores 0.9928483088670216
========================Fold5==========================

Validation scores 0.9145921103025662

Training scores 0.9924213920988981
========================Fold6==========================

Validation scores 0.919557758292032

Training scores 0.9912818856958348
========================Fold7==========================

Validation scores 0.908813429988554

Training scores 0.9933326164103666
========================Fold8==========================

Validation scores 0.924642049736247

In [911]:
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
gnb_model = GaussianNB()
gnb_train, gnb_test, gnb_name = model(gnb_model,X,y,test_df.drop(['target'], axis =1),'GNB(1)')

========================Fold1==========================

Validation scores 0.7577965607694549

Training scores 0.75682823023842
========================Fold2==========================

Validation scores 0.7527440785673022

Training scores 0.7568978374347501
========================Fold3==========================

Validation scores 0.757699012202208

Training scores 0.7557672395941188
========================Fold4==========================

Validation scores 0.7524467472654001

Training scores 0.7569024127352624
========================Fold5==========================

Validation scores 0.7533217793183131

Training scores 0.7573209625978545
========================Fold6==========================

Validation scores 0.754041570438799

Training scores 0.7571440404207737
========================Fold7==========================

Validation scores 0.763986013986014

Training scores 0.755959512497418
========================Fold8==========================

Validation scores 0.7595231171852282

T

In [912]:
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
log_model = LogisticRegression()
log_train, log_test, log_name = model(log_model,X,y,test_df.drop(['target'], axis =1),'LOG(1)')

========================Fold1==========================

Validation scores 0.7509293680297398

Training scores 0.7413282369804108
========================Fold2==========================

Validation scores 0.7370140476968311

Training scores 0.75
========================Fold3==========================

Validation scores 0.7404453306746428

Training scores 0.7467145693485865
========================Fold4==========================

Validation scores 0.7500812479688007

Training scores 0.7528552705485864
========================Fold5==========================

Validation scores 0.7397534155281571

Training scores 0.7464528069093154
========================Fold6==========================

Validation scores 0.7469642271086314

Training scores 0.751412429378531
========================Fold7==========================

Validation scores 0.7455386649041639

Training scores 0.7466503113795055
========================Fold8==========================

Validation scores 0.7644503842298698

Training s

In [913]:
from sklearn.linear_model import SGDClassifier
sgd_model = SGDClassifier(loss="log",penalty="l2", max_iter=500)
sgd_train, sgd_test, sgd_name = model(sgd_model,X,y,test_df.drop(['target'], axis =1),'SGD(1)')

========================Fold1==========================

Validation scores 0.7600459242250288

Training scores 0.7554018311291962
========================Fold2==========================

Validation scores 0.587917042380523

Training scores 0.5790881221055378
========================Fold3==========================

Validation scores 0.7556066705002876

Training scores 0.755351053777577
========================Fold4==========================

Validation scores 0.556024378809189

Training scores 0.5363105741821637
========================Fold5==========================

Validation scores 0.7536973833902162

Training scores 0.756120412236751
========================Fold6==========================

Validation scores 0.49688149688149685

Training scores 0.4807391401534908
========================Fold7==========================

Validation scores 0.5186660029865605

Training scores 0.5180205949656751
========================Fold8==========================

Validation scores 0.7570653725378247

In [914]:
from sklearn.ensemble import RandomForestClassifier
rfm_model = RandomForestClassifier(n_estimators = 2000, oob_score=True, n_jobs=-1, max_features=None, min_samples_leaf=30)
rfm_train, rfm_test, rfm_name = model(rfm_model,X,y,test_df.drop(['target'], axis =1),'RFM(1)')

========================Fold1==========================

Validation scores 0.8909606809890555

Training scores 0.90938380990737
========================Fold2==========================

Validation scores 0.9015334947538337

Training scores 0.9103559312287965
========================Fold3==========================

Validation scores 0.9063761097659403

Training scores 0.9095303867403315
========================Fold4==========================

Validation scores 0.8925820835022295

Training scores 0.9098722282702115
========================Fold5==========================

Validation scores 0.8964127367996776

Training scores 0.911243964129685
========================Fold6==========================

Validation scores 0.9087974172719936

Training scores 0.90938380990737
========================Fold7==========================

Validation scores 0.8981744421906693

Training scores 0.9126146788990825
========================Fold8==========================

Validation scores 0.915576694411415

T

In [915]:
from sklearn.ensemble import AdaBoostClassifier
ada_model = RandomForestClassifier(n_estimators = 2000, random_state=101)
ada_train, ada_test, ada_name = model(ada_model,X,y,test_df.drop(['target'], axis =1),'ADA')

========================Fold1==========================

Validation scores 0.918733307897749

Training scores 0.9927949241853963
========================Fold2==========================

Validation scores 0.9256576439191765

Training scores 0.9926300500295875
========================Fold3==========================

Validation scores 0.9241849886277482

Training scores 0.9926850258175559
========================Fold4==========================

Validation scores 0.9272588055130168

Training scores 0.992844461182547
========================Fold5==========================

Validation scores 0.9239005736137668

Training scores 0.9924132364810332
========================Fold6==========================

Validation scores 0.9326702371843917

Training scores 0.9925224595190703
========================Fold7==========================

Validation scores 0.918007662835249

Training scores 0.9933275936289281
========================Fold8==========================

Validation scores 0.9361058601134216

In [916]:
from sklearn.tree import DecisionTreeClassifier
dtc_model = DecisionTreeClassifier(max_depth=8,min_samples_leaf=5,criterion = 'entropy',min_samples_split=10,random_state=10)
dtc_train, dtc_test, dtc_name= model(dtc_model,X,y,test_df.drop(['target'], axis =1), 'dtc')

========================Fold1==========================

Validation scores 0.7959565705728191

Training scores 0.8104485488126649
========================Fold2==========================

Validation scores 0.7964796479647964

Training scores 0.8052938062466914
========================Fold3==========================

Validation scores 0.8046407185628742

Training scores 0.8077231641439804
========================Fold4==========================

Validation scores 0.8032044583768722

Training scores 0.8169576059850375
========================Fold5==========================

Validation scores 0.7930283224400871

Training scores 0.8100753051155545
========================Fold6==========================

Validation scores 0.811908931698774

Training scores 0.8146560319042871
========================Fold7==========================

Validation scores 0.8142153413089375

Training scores 0.8137128072445019
========================Fold8==========================

Validation scores 0.81520223152022

In [917]:
/

()

In [918]:
def xgb(model,train,label,test,model_name):
    mean_train = []
    mean_test = []
    test_pred = np.zeros(test.shape[0])
    val_pred = np.zeros(train.shape[0])
    for count, (train_index,test_index) in enumerate(skf.split(train,label)):
        x_train,x_test = train.iloc[train_index],train.iloc[test_index]
        y_train,y_test = label.iloc[train_index],label.iloc[test_index]
        print(f'========================Fold{count +1}==========================')
        model.fit(x_train, y_train, early_stopping_rounds = 500, eval_metric="auc",
                           eval_set=[(x_test, y_test)],verbose=250)
        train_predict = model.predict(x_train, ntree_limit = model.get_booster().best_ntree_limit)
        test_predict = model.predict(x_test, ntree_limit = model.get_booster().best_ntree_limit)
        val_pred[test_index] = test_predict
        test_pred+= model.predict(test, ntree_limit = model.get_booster().best_ntree_limit)
        
        print('\nTesting scores', f1_score(y_test,test_predict))
        print('\nTraining scores', f1_score(y_train,train_predict))
        mean_train.append(f1_score(y_train, train_predict))
        mean_test.append(f1_score(y_test,test_predict))
    print('Average Testing ROC score for 10 folds split:',np.mean(mean_test))
    print('Average Training ROC score for 10 folds split:',np.mean(mean_train))
    print('standard Deviation for 10 folds split:',np.std(mean_test))
    return val_pred, test_pred, model_name

In [919]:
from xgboost import XGBClassifier
Xgboost = XGBClassifier(learning_rate=0.5,subsample=0.7,colsample_bytree=0.9,reg_alpha=6,
               n_jobs=-1,n_estimators=5000,max_depth= 7,random_state=34)

xgb_train, xgb_test, xgb_name= xgb(Xgboost,X,y,test_df.drop(['target'], axis =1),'xgboost')

========================Fold1==========================
[0]	validation_0-auc:0.88633
Will train until validation_0-auc hasn't improved in 500 rounds.
[250]	validation_0-auc:0.97227
[500]	validation_0-auc:0.97176
Stopping. Best iteration:
[86]	validation_0-auc:0.97268


Testing scores 0.9199066874027994

Training scores 0.949005666037107
========================Fold2==========================
[0]	validation_0-auc:0.88508
Will train until validation_0-auc hasn't improved in 500 rounds.
[250]	validation_0-auc:0.97194
[500]	validation_0-auc:0.97095
Stopping. Best iteration:
[100]	validation_0-auc:0.97307


Testing scores 0.9198443579766536

Training scores 0.9498971250625591
========================Fold3==========================
[0]	validation_0-auc:0.88007
Will train until validation_0-auc hasn't improved in 500 rounds.
[250]	validation_0-auc:0.97457
[500]	validation_0-auc:0.97376
Stopping. Best iteration:
[77]	validation_0-auc:0.97581


Testing scores 0.9236165237724083

Training scores

In [920]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hist_model = HistGradientBoostingClassifier(max_iter = 3000, learning_rate = 0.6, loss='auto',random_state = 1)
hist_train, hist_test, hist_name= model(hist_model,X,y,test_df.drop(['target'], axis =1), 'hist')

========================Fold1==========================

Validation scores 0.9133307423241352

Training scores 0.950480822372057
========================Fold2==========================

Validation scores 0.905320813771518

Training scores 0.9222247090420769
========================Fold3==========================

Validation scores 0.9175738724727837

Training scores 0.9373991430638251
========================Fold4==========================

Validation scores 0.9071290944123314

Training scores 0.9528166685040238
========================Fold5==========================

Validation scores 0.9081751259201859

Training scores 0.9367313019390582
========================Fold6==========================

Validation scores 0.9218139171227522

Training scores 0.9319120437548833
========================Fold7==========================

Validation scores 0.9174168297455968

Training scores 0.9369832402234637
========================Fold8==========================

Validation scores 0.921658986175115

In [921]:
Stack_Train = pd.concat([pd.DataFrame(LGB1_train), pd.DataFrame(gbm_train), pd.DataFrame(ada_train),pd.DataFrame(xgb_train),
                         pd.DataFrame(dtc_train), pd.DataFrame(rfm_train), pd.DataFrame(sgd_train), pd.DataFrame(gnb_train),
                         pd.DataFrame(log_train),pd.DataFrame(hist_train)],1)

Stack_Test = pd.concat([pd.DataFrame(LGB1_test), pd.DataFrame(gbm_test), pd.DataFrame(ada_test),pd.DataFrame(xgb_test),
                         pd.DataFrame(dtc_test), pd.DataFrame(rfm_test), pd.DataFrame(sgd_test), pd.DataFrame(gnb_test),
                         pd.DataFrame(log_test),pd.DataFrame(hist_test)],1)

Stack_Train.columns=[LGB1_name,gbm_name,ada_name,xgb_name,dtc_name,rfm_name,sgd_name,gnb_name,log_name,hist_name]
Stack_Test.columns=[LGB1_name,gbm_name,ada_name,xgb_name,dtc_name,rfm_name,sgd_name,gnb_name,log_name,hist_name]

Stack_Test = Stack_Test/8


In [922]:
Stack_Test.head()

,lightgbm(1),Gbm,ADA,xgboost,dtc,RFM(1),SGD(1),GNB(1),LOG(1),hist
0,1.0,0.000,0.0,0.375,0.500,0.875,0.50,1.0,0.0,0.250
1,1.0,1.000,0.0,1.000,0.500,1.000,0.50,1.0,1.0,0.375
2,1.0,1.000,0.0,1.000,1.000,1.000,0.50,1.0,0.0,1.000
3,0.0,0.375,0.0,0.250,0.000,0.000,0.00,0.0,0.0,0.125
4,1.0,0.000,0.0,1.000,0.875,0.750,0.75,1.0,1.0,1.000


In [923]:
from sklearn.linear_model import LogisticRegression, LinearRegression
base_model = LogisticRegression()
Stacking(base_model,Stack_Train,Stack_Test,y,'ffinal_stack11.csv')

In [ ]:
from sklearn.linear_model import LogisticRegression, LinearRegression
base_model = SVC(class_weight = 'balanced', probability = True)
Stacking(base_model,Stack_Train,Stack_Test,y,'ffinal_stack13.csv')

In [930]:
from imblearn.ensemble import BalancedBaggingClassifier
bbc = BalancedBaggingClassifier(base_estimator= gbm_model, sampling_strategy='auto',replacement = False, random_state = 1)

In [ ]:
b_model = bbc.fit(X_train,y_train)
pred = b_model.predict(X_test)
print(f1_score(y_test,pred))

In [ ]:
preds = b_model.predict(test_df.drop(['target'],axis =1))
sub= ttest[["ID"]]
sub['target'] = preds
sub.to_csv('catd.csv', index = False)